<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Mecanismos de auto-atención</center></span>

<figure>
<center>
<img src="../Imagenes/child_engaging_in_joint_attention.jpg" width="800" height="600" align="center"/>
</center>
<figcaption>
</figcaption>
</figure>

Fuente: <a href="https://commons.wikimedia.org/wiki/File:Illustration_of_caregiver_and_child_engaging_in_joint_attention.jpg">Rita.obeid6</a>, <a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a>, via Wikimedia Commons

##   <span style="color:blue">Autores</span>

- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Referencias</span>

1. [Illustrated Auto-attención](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)
1. [Illustrated Attention](https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3#0458)
1. [Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et. al, 2015)](https://arxiv.org/pdf/1409.0473.pdf)
1. [Effective Approaches to Attention-based Neural Machine Translation (Luong et. al, 2015)](https://arxiv.org/pdf/1508.04025.pdf)
1. [Attention Is All You Need (Vaswani et. al, 2017)](https://arxiv.org/pdf/1706.03762.pdf)
1. [Self-Attention GAN (Zhang et. al, 2018)](https://arxiv.org/pdf/1805.08318.pdf)
1. [Sequence to Sequence Learning with Neural Networks (Sutskever et. al, 2014)](https://arxiv.org/pdf/1409.3215.pdf)
1. [TensorFlow’s seq2seq Tutorial with Attention (Tutorial on seq2seq+attention)](https://github.com/tensorflow/nmt)
1. [Lilian Weng’s Blog on Attention (Great start to attention)](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#a-family-of-attention-mechanisms)
1. [Jay Alammar’s Blog on Seq2Seq with Attention (Great illustrations and worked example on seq2seq+attention)](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/)
1. [Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation (Wu et. al, 2016)](https://arxiv.org/pdf/1609.08144.pdf)


## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Modelo seq2seq](#Modelo-seq2seq)
* [Auto-atención](#Auto-atención)
    * [Paso 1. Preparar las entradas](#Paso-1.-Preparar-las-entradas)
    * [Paso 2. Inicializar los pesos](#Paso-2.-Inicializar-los-pesos)
    * [Paso 3. Obtener claves, consultas y valores](#Paso-3.-Obtener-claves,-consultas-y-valores)
    * [Paso 4. Cálculo de los puntajes de auto-atención para las entradas](#Paso-4.-Cálculo-de-los-puntajes-de-auto-atención-para-las-entradas)
    * [Paso 5. Cálculo del puntaje softmax](#Paso-5.-Cálculo-del-puntaje-softmax)
    * [Paso 6. Multiplica los puntajes softmax con los valores para cada consulta](#Paso-6.-Multiplica-los-puntajes-softmax-con-los-valores-para-cada-consulta)
    * [Paso 7. Suma pesada de valores para conseguir los vectores de contexto](#Paso-7.-Suma-pesada-de-valores-para-conseguir-los-vectores-de-contexto)
    * [Ejemplo de implementación en Python](#Ejemplo-de-implementación-en-Python)


## <span style="color:blue">Introducción</span>

¿Qué tienen en común BERT, RoBERTa, ALBERT, SpanBERT, DistilBERT, SesameBERT, SemBERT, SciBERT, BioBERT, MobileBERT, TinyBERT y CamemBERT?. NO es propiamente BERT.

Respuesta: auto-atención (`self-attention`)🤗. No solo estamos hablando de arquitecturas que llevan el nombre "BERT", sino más correctamente de arquitecturas basadas en transformers. 

Las arquitecturas basadas en transformers que se utilizan principalmente en el modelado de tareas de comprensión del lenguaje, evitan el uso de la recurrencia en la red neuronal y, en cambio, confían por completo en los mecanismos de auto-atención para generar dependencias globales entre entradas y salidas. 


La auto-atención es similar a la atención. Comparten fundamentalmente el mismo concepto y muchas operaciones matemáticas comunes.  Un módulo de auto-atención toma $n$ entradas y devuelve $n$ salidas. ¿Qué pasa en este módulo? En términos sencillos, el mecanismo de auto atención permite que las entradas interactúen entre sí (`auto`) y descubran a quién deben prestar más atención (`atención`). Los resultados son agregados de estas interacciones y puntuaciones de atención.


En esta lección  revisamos los detalles de construcción de los mecanismos de atención.

Supongamos que tenemos un secuencia con tres entradas $e_1, e_2$ y $e_3$. Un mecanismo de atención siguen el siguiente algoritmo.



1. Prepara las entradas
1. Inicializa los pesos
1. Obtiene `clave` (key), `consulta` (query) y `valor` (value) a aprtir de las entradas.
1. Calcula las puntuaciones de atención para la entrada $e_1$.
1. Calcula el softmax de las puntuaciones.
1. Multiplica las puntuaciones softmax con los `valores`
1. Sumar `valores ponderados` para obtener la Salida 1
1. Repite los pasos 4 a 7 para la entrada $e_2$ y la entrada $e_3$



## <span style="color:blue">Auto-atención</span>

### <span style="color:#4CC9F0">Paso 1. Preparar las entradas</span>

Las entradas corresponden a una secuencia de palabras sumergidas (embebidas) en un sumergimiento (embbeding) el cual vamos a suponer de tamaño 4. Para el ejemplo supondremos que la secuencia consta de 3 palabras. Entonces tenemos 3 entradas de tamaño 4.

<figure>
<center>
<img src="../Imagenes/self_attention_1.gif" width="800" height="800" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Auto-atención: Entradas.</p>
</figcaption>
</figure>


Fuente: [Illustrated self attention](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)

En el ejemplo tenemos tres entradas.

1. entrada 1: $e_1=[1, 0, 1, 0]$
1. entrada 2: $e_2=[0, 2, 0, 2]$
1. entrada 3: $e_3=[1, 1, 1, 1]$

En notación matricial escribimos

$$
E = \begin{pmatrix}
1 & 0 & 1 & 0\\
0 & 2 & 0 & 2\\
1 & 1 & 1 &1
\end{pmatrix}= \begin{pmatrix} 
e_1\\
e_2\\
e_3\\
\end{pmatrix}.
$$

### <span style="color:#4CC9F0">Paso 2. Inicializar los pesos</span>

Cada una de las entradas se transforman en tres diferentes vectores de algún tamaño. Para ello se construyen tres transformaciones lineales (o afines), digamos $W_k$, $W_q$ y $W_v$. Esta matrices son parámetros de la red neuronal que son inicializados aleatoriamente, o con algún procedimiento estándar  para  inicializar  los  pesos.

Para esta ilustración, supondremos que las matrices de pesos son dadas por



$$
W_k = 
\begin{pmatrix} 
0 & 0 &1\\
1 & 1 &0\\
0 & 1 &0\\
1 & 1 &0
\end{pmatrix}, \quad
W_v = 
\begin{pmatrix} 
0 & 2 &0\\
0 & 3 &0\\
1 & 0 &3\\
1 & 1 &0
\end{pmatrix}, \quad
W_q = 
\begin{pmatrix} 
1 & 0 &1\\
1 & 0 &0\\
0 & 0 &1\\
0 & 1 &1
\end{pmatrix}.
$$


### <span style="color:#4CC9F0">Paso 3. Obtener claves, consultas y valores</span>

En este paso obtenemos los tres tipos de objetos derivados de la entrada: claves, consultas y valores.

#### Claves (keys)

Las claves se obtienen de la siguiente manera:

$$
K = E \times W_k = 
\begin{pmatrix} 
1 & 0 &1 &0\\
0 & 2 &0 &2\\
1 & 1 &1 &1\\
\end{pmatrix} \times
\begin{pmatrix} 
0 & 0 &1\\
1 & 1 &0\\
0 & 1 &0\\
1 & 1 &0
\end{pmatrix}  =
\begin{pmatrix} 
0 & 1 &1\\
4 & 4 &0 \\
2 & 3 &1 \\
\end{pmatrix}=
\begin{pmatrix} 
k_1\\
k_2\\
k_3\\
\end{pmatrix}.
$$

#### Valores (values)

Los valores se obtiene como

$$
V = E \times W_v = 
\begin{pmatrix} 
1 & 0 &1 &0\\
0 & 2 &0 &2\\
1 & 1 &1 &1\\
\end{pmatrix} \times
\begin{pmatrix} 
0 & 2 &0\\
0 & 3 &0\\
1 & 0 &3\\
1 & 1 &0
\end{pmatrix}   =
\begin{pmatrix} 
1 & 2 &3\\
2 & 8 &0 \\
2 & 6 &3 \\
\end{pmatrix}=
\begin{pmatrix} 
v_1\\
v_2\\
v_3\\
\end{pmatrix}.
$$


#### Consultas (queries)

Finalmente las consultas resultantes son

$$
Q = E \times W_q = 
\begin{pmatrix} 
1 & 0 &1 &0\\
0 & 2 &0 &2\\
1 & 1 &1 &1\\
\end{pmatrix} \times
\begin{pmatrix} 
1 & 0 &1\\
1 & 0 &0\\
0 & 0 &1\\
0 & 1 &1
\end{pmatrix}   =
\begin{pmatrix} 
1 & 0 &2\\
2 & 2 &2 \\
2 & 1 &3 \\
\end{pmatrix}=
\begin{pmatrix} 
q_1\\
q_2\\
q_3\\
\end{pmatrix}.
$$


<figure>
<center>
<img src="../Imagenes/self_attention_2.png" width="800" height="800" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Auto-atención: Obtención de claves consultas y valores.</p>
</figcaption>
</figure>

Fuente: [Illustrated self attention](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)

### <span style="color:#4CC9F0">Paso 4. Cálculo de los puntajes de auto-atención para las entradas</span>

Ilustramos aquí como se calculan los puntajes de auto-atención para la entrada 1. Los puntajes de auto-atención para las demás entradas se calculan de la misma manera con el cambio obvio.

<figure>
<center>
<img src="../Imagenes/self_attention_3.png" width="800" height="800" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Auto-atención: Obtención de puntajes de auto-atención para la primera consulta.</p>
</figcaption>
</figure>

Fuente: [Illustrated self attention](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)

Como se muestra en la anterior ilustración, se toma la consulta obtenida para la primera entrada, que en el ejemplo es $e_1 =[1, 0, 2]$. Los pesos de auto-atención son una medida de similaridad, entre la consulta y cada una de las claves, como se ha estudiado en las lecciones atención. La función de auto-atención se denotará $a$ . En este caso el resultado se obtiene haciendo el producto escalar (producto punto) entre la consulta asociada  a $e_1$, es decir, $q_1$ y cada una de las claves, es decir $a(q_1, c_i)= <q_1, c_i>, i =1,2,3$.

Escrito en forma matricial, los pesos de auto-atención para la entrada 1 se calculan como


$$
p_1 = q_1 \times K^T = [1, 0, 2]\begin{pmatrix} 
0 & 4 & 2 \\
1 & 4 & 3 \\
1 & 0 & 1 \\
\end{pmatrix} =[2, 4, 4].
$$

Los pesos de auto-atención completos, es decir para todas las  consultas  se calculan mediante

$$
P = Q \times  K^T =
\begin{pmatrix} 
1 & 0 &2\\
2 & 2 &2 \\
2 & 1 &3 \\
\end{pmatrix} \times
\begin{pmatrix} 
0 & 4 & 2 \\
1 & 4 & 3 \\
1 & 0 & 1 \\
\end{pmatrix}= \begin{pmatrix} 
2 & 4 & 4 \\
4 & 16 & 12 \\
4 & 12 & 10 \\
\end{pmatrix} = \begin{pmatrix} 
p_1\\
p_2\\
p_3\\
\end{pmatrix}
$$

### <span style="color:#4CC9F0">Paso 5. Cálculo del puntaje softmax</span>

Transformamos los puntajes a la escala softmax. Por ejemplo softmax([2, 4, 4]) = [0.06, 0.47, 0.47]. Observe que softmax define una distribución discreta de probabilidad. Ele cálculo completo para el ejemplo es


$$
S = \begin{pmatrix}
\text{softmax}(p_1)\\
\text{softmax}(p_2)\\
\text{softmax}(p_3)
\end{pmatrix} = 
\begin{pmatrix} 
0.06 & 0.47 &0.47\\
0.00 & 0.98 &0.02 \\
0.00 & 0.88 &0.12 \\
\end{pmatrix} =
\begin{pmatrix}
s_1\\
s_2\\
s_3
\end{pmatrix}
$$

### <span style="color:#4CC9F0">Paso 6. Multiplica los puntajes softmax con los valores para cada consulta</span>

Para la consulta 1 se tiene
* 0.06 * [1, 2, 3] = [0.06, 0.13, 0.19]
* 0.47 * [2, 8, 0] = [0.94, 3.75, 0.00]
* 0.47 * [2, 6, 3] = [0.94, 2.81, 1.40]

Para la consulta 2
* 0.00 * [1, 2, 3] = [0.00, 0.00, 0.00]
* 0.98 * [2, 8, 0] = [1.96, 7.86, 0.00]
* 0.02 * [2, 6, 3] = [0.04, 0.11, 0.05]

Para la consulta 3
* 0.00 * [1, 2, 3] = [0.00, 0.00, 0.00]
* 0.88 * [2, 8, 0] = [1.76, 7.04, 0.00]
* 0.12 * [2, 6, 3] = [0.24, 0.72, 0.36]



### <span style="color:#4CC9F0">Paso 7. Suma pesada de valores para conseguir los vectores de contexto</span>

Cada salida (vector de contexto) se obtiene como la suma de los vectores pesados asociados a cada consulta así:

* salida 1: [0.06, 0.13, 0.19] + [0.94, 3.75, 0.00] + [0.94, 2.81, 1.40] = [1.94, 6.68, 1.60]
* salida 2: [0.00, 0.00, 0.00] + [1.96, 7.86, 0.00] + [0.04, 0.11, 0.05] = [2.00, 7.96, 0.05]
* salida 3: [0.00, 0.00, 0.00] + [1.76, 7.04, 0.00] + [0.24, 0.72, 0.36] = [2.00, 7.76, 0.36]

La siguiente imagen muestra el modelo completo de auto-atención. Los valores se redondearon. 

<figure>
<center>
<img src="../Imagenes/self_attention_4.png" width="800" height="800" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Auto-atención: Cálculo completo de las últimas dos salidas</p>
</figcaption>
</figure>

Fuente: [Illustrated self attention](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)

### <span style="color:#4CC9F0">Ejemplo de implementación en Python</span>

In [61]:
#@Auto-atención
import numpy as np
from scipy.special import softmax

# Entradas
# E
E = np.array([[1, 0, 1, 0],
              [0, 2, 0, 2],
              [1, 1, 1, 1]])

# Matrices de pesos W_k, W_v y W_q
# W_k
W_k = np.array([[0, 0, 1],
                [1, 1, 0],
                [0, 1, 0],
                [1, 1, 0]])

# W_v
W_v = np.array([[0, 2, 0],
                [0, 3, 0],
                [1 ,0, 3],
                [1, 1, 0]])

# W_q
W_q = np.array([[1, 0, 1],
                [1, 0, 0],
                [0, 0, 1],
                [0, 1, 1]])

# Claves (K), Valores (V), Consultas (Q)
# K
K = E @ W_k

# V
V = E @ W_v

# Q
Q = E @ W_q

# Puntajes
# P
P = Q @ np.transpose(K)

# Puntajes softmax
# S
S = softmax(P, axis=1)

# Valores pesados con los puntajes softmax
# valor 1
#v0 = (np.transpose(np.tile(np.transpose(S)[:,0], (3,1))))*V
# valor 2
#v1 = (np.transpose(np.tile(np.transpose(S)[:,1], (3,1))))*V
# valor 3
#v2 = (np.transpose(np.tile(np.transpose(S)[:,2], (3,1))))*V

wV = V[:,None] * S.T[:,:,None]

# Salidas:  sumas de los valores pesados
#o0=  np.sum(v0, axis=0)
#o1=  np.sum(v1, axis=0)
#o2=  np.sum(v2, axis=0)

O = wV.sum( axis=0)

print('E=\n ', E)

print('W_k\n', W_k)
print('W_v\n', W_v)
print('W_q\n', W_q)

print('K\n', K)
print('V\n', V)
print('Q\n', Q)
print('P\n', P)
print('S\n', np.round(S,2))

#print('v0:', np.round(v0,2))
#print('v1:', np.round(v1,2))
#print('v2:', np.round(v2,2))

#print('o0:', np.round(o0,2))
#print('o1:', np.round(o1,2))
#print('o2:', np.round(o2,2))

print('wV\n',np.round(wV,2))
print('O\n',np.round(O,2))

E=
  [[1 0 1 0]
 [0 2 0 2]
 [1 1 1 1]]
W_k
 [[0 0 1]
 [1 1 0]
 [0 1 0]
 [1 1 0]]
W_v
 [[0 2 0]
 [0 3 0]
 [1 0 3]
 [1 1 0]]
W_q
 [[1 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 1]]
K
 [[0 1 1]
 [4 4 0]
 [2 3 1]]
V
 [[1 2 3]
 [2 8 0]
 [2 6 3]]
Q
 [[1 0 2]
 [2 2 2]
 [2 1 3]]
P
 [[ 2  4  4]
 [ 4 16 12]
 [ 4 12 10]]
S
 [[0.06 0.47 0.47]
 [0.   0.98 0.02]
 [0.   0.88 0.12]]
wV
 [[[0.06 0.13 0.19]
  [0.   0.   0.  ]
  [0.   0.   0.  ]]

 [[0.94 3.75 0.  ]
  [1.96 7.86 0.  ]
  [1.76 7.04 0.  ]]

 [[0.94 2.81 1.4 ]
  [0.04 0.11 0.05]
  [0.24 0.72 0.36]]]
O
 [[1.94 6.68 1.6 ]
 [2.   7.96 0.05]
 [2.   7.76 0.36]]
